In [2]:

from litellm import completion
import time
gpt_model = "o4-mini"
#claude_model = "claude-opus-4-20250514"
claude_model = "claude-sonnet-4-20250514"
#claude_model = "o4-mini"
import os

import chess
board = chess.Board()

import chess.svg
from IPython.display import display, SVG, clear_output



gpt_system = """You are a grandmaster chess player named PLAYER_GPT playing White.

PRIORITIES (in order):
1. If you can deliver checkmate, DO IT immediately
2. If opponent's king is exposed, attack aggressively with all available pieces
3. Capture undefended pieces — especially high-value ones (Queen > Rook > Bishop/Knight > Pawn)
4. Never leave your pieces undefended — check if your move hangs a piece
5. Control the center, develop pieces quickly, castle early
6. Create forks, pins, and skewers whenever possible
7. Push for checkmate — do NOT shuffle pieces aimlessly

AVOID:
- Moving the same piece twice in the opening
- Moving pawns when you should be developing pieces
- Trading pieces when you have an advantage
- Pointless moves that don't improve your position

Play AGGRESSIVELY. Aim to win in as few moves as possible.

You will receive the current board state and a list of legal moves.
Respond with ONLY your next move in Standard Algebraic Notation (SAN).
Output ONLY the move. No explanation, no board, no commentary.
Example valid output: e4
Example valid output: Nf3"""

claude_system = """You are a grandmaster chess player named PLAYER_CLAUDE playing Black.

PRIORITIES (in order):
1. If you can deliver checkmate, DO IT immediately
2. If opponent's king is exposed, attack aggressively with all available pieces
3. Capture undefended pieces — especially high-value ones (Queen > Rook > Bishop/Knight > Pawn)
4. Never leave your pieces undefended — check if your move hangs a piece
5. Control the center, develop pieces quickly, castle early
6. Create forks, pins, and skewers whenever possible
7. Push for checkmate — do NOT shuffle pieces aimlessly

AVOID:
- Moving the same piece twice in the opening
- Moving pawns when you should be developing pieces
- Trading pieces when you have an advantage
- Pointless moves that don't improve your position

Play AGGRESSIVELY. Aim to win in as few moves as possible.

You will receive the current board state and a list of legal moves.
Respond with ONLY your next move in Standard Algebraic Notation (SAN).
Output ONLY the move. No explanation, no board, no commentary.
Example valid output: e5
Example valid output: Nf6"""

gpt_messages = []
claude_messages = []
move_count=0
illegal_count=0

from IPython.display import display, HTML, clear_output


import matplotlib.pyplot as plt
import base64
from io import BytesIO


metrics = {
    "move_number": [],
    "player": [],
    "context_size": [],
    "illegal_cumulative": [],
}


def update_graph():
    if len(metrics["move_number"]) < 2:
        return ""

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 4))

    ax1.plot(metrics["move_number"], metrics["context_size"], color="red", linewidth=2)
    ax1.set_title("Context Size Per Move", fontsize=10)
    ax1.set_xlabel("Move")
    ax1.set_ylabel("Characters")
    ax1.grid(True, alpha=0.3)

    ax2.plot(metrics["move_number"], metrics["illegal_cumulative"], color="orange", linewidth=2)
    ax2.set_title("Cumulative Illegal Attempts", fontsize=10)
    ax2.set_xlabel("Move")
    ax2.set_ylabel("Illegal Attempts")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    buf = BytesIO()
    fig.savefig(buf, format='png', dpi=100)
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    return f'<img src="data:image/png;base64,{img_base64}" />'


def call_gpt(gpt_system, gpt_messages, claude_messages):
    global move_count, illegal_count
    messages = [{"role": "system", "content": gpt_system}]
    #legal_moves = [board.san(m) for m in board.legal_moves]
    if claude_messages:
        for g, c in zip(gpt_messages, claude_messages):
            messages.append({"role": "assistant", "content": g})
            messages.append({"role": "user", "content": c})
    else:
        messages.append({"role": "user", "content": str(board)})
    messages.append({"role": "user", "content": f"Current board:\n{str(board)}\n\nPick the best move."})
    #messages.append({"role": "user", "content": f"Current board:\n{str(board)}\n\nYour legal moves: {', '.join(legal_moves)}\n\nPick the best move."})

    move_count += 1
    input_tokens = sum(len(m["content"]) for m in messages)

    for attempt in range(10):
        response = completion(model=gpt_model, messages=messages)
        move = move = response.choices[0].message.content.strip().split('\n')[-1].split()[-1]
        try:
            board.push_san(move)
            gpt_messages.append(str(board))

            # Track metrics
            metrics["move_number"].append(move_count)
            metrics["player"].append("GPT")  # or "Claude"
            metrics["context_size"].append(input_tokens)
            metrics["illegal_cumulative"].append(illegal_count)

            clear_output(wait=True)
            svg_str = chess.svg.board(board, lastmove=board.peek(), size=350)
            graph_html = update_graph()
            stats = f"""
            <div style="font-family: monospace; font-size: 14px; margin-bottom: 10px;">
                 <b>♟️ WITH HISTORY</b><br>
                 Move {move_count} | GPT plays: {move}<br>
                 Context size: ~{input_tokens} chars | History messages: {len(messages)}<br>
                 Illegal attempts so far: {illegal_count}<br>
            </div>
            """
            layout = f"""
            {stats}
            <div style="display: flex; align-items: margin-top: -20x; gap: 20px;">
                <div>{svg_str}</div>
                <div>{graph_html}</div>
            </div>
            """
            display(HTML(layout))
            if board.is_game_over():
                return False
            return True
        except Exception as e:
            illegal_count += 1
            print(f"GPT invalid move '{move}' (attempt {attempt+1}): {e}")
            messages.append({"role": "assistant", "content": move})
            messages.append({"role": "user", "content": f"'{move}' is illegal. Try again. Current board:\n{str(board)}"})
    print("GPT failed 10 attempts, ending game.")
    return False

def call_claude(claude_system, gpt_messages, claude_messages):
    global move_count, illegal_count
    messages = [{"role": "system", "content": claude_system}]
    #legal_moves = [board.san(m) for m in board.legal_moves]
    for g, c in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": g})
        messages.append({"role": "assistant", "content": c})
    messages.append({"role": "user", "content": f"{gpt_messages[len(claude_messages)]}\n\nPick the best move."})

    move_count += 1
    input_tokens = sum(len(m["content"]) for m in messages)

    for attempt in range(10):
        response = completion(model=claude_model, messages=messages)
        move = response.choices[0].message.content.strip().split('\n')[-1].split()[-1]
        try:
            board.push_san(move)
            claude_messages.append(str(board))
            # Track metrics
            metrics["move_number"].append(move_count)
            metrics["player"].append("Claude")
            metrics["context_size"].append(input_tokens)
            metrics["illegal_cumulative"].append(illegal_count)

            clear_output(wait=True)
            svg_str = chess.svg.board(board, lastmove=board.peek(), size=350)
            graph_html = update_graph()
            stats = f"""
            <div style="font-family: monospace; font-size: 14px; margin-bottom: 10px;">
                 <b>♟️ WITH HISTORY</b><br>
                 Move {move_count} | GPT plays: {move}<br>
                 Context size: ~{input_tokens} chars | History messages: {len(messages)}<br>
                 Illegal attempts so far: {illegal_count}<br>
            </div>
            """
            layout = f"""
            {stats}
            <div style="display: flex; align-items: margin-top: -20x; gap: 20px;">
                <div>{svg_str}</div>
                <div>{graph_html}</div>
            </div>
            """
            display(HTML(layout))
            if board.is_game_over():
                return False
            return True
        except Exception as e:
            illegal_count += 1
            print(f"Claude invalid move '{move}' (attempt {attempt+1}): {e}")
            messages.append({"role": "assistant", "content": move})
            messages.append({"role": "user", "content": f"'{move}' is illegal. Try again. Current board:\n{str(board)}"})
    print("Claude failed 10 attempts, ending game.")
    return False


while True:
    if not call_gpt(gpt_system, gpt_messages, claude_messages):
        break
    time.sleep(5)
    if not call_claude(claude_system, gpt_messages, claude_messages):
        break
    time.sleep(5)
# After the while loop ends:
print()
if board.is_checkmate():
    winner = "Black (Claude)" if board.turn == chess.WHITE else "White (GPT)"
    print(f"♚ CHECKMATE! {winner} wins!")
elif board.is_stalemate():
    print("½ STALEMATE! Draw.")
elif board.is_insufficient_material():
    print("½ DRAW — Insufficient material.")
else:
    print("Game ended.")
print(f"Total moves: {move_count} | Total illegal attempts: {illegal_count}")


ModuleNotFoundError: No module named 'matplotlib'